In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import pandas as pd
import un_transforms as tr

#importing the dataset
mixed = pd.read_csv("./data/sup_final_copy2.csv", sep=",", parse_dates=[0,3,5,13,14])

In [2]:
#importing the attribute transformer class
import attributeTransformer as attrTr
from sklearn import metrics

In [3]:
#Rule Based Algorithm - CN2Learner
import orangedf
import Orange as orange
from Orange.data import Table, Domain, ContinuousVariable, DiscreteVariable

newtrFC = attrTr.TransformFeatureColumns(mixed)
newtrFC.splitData(0.2)

In [84]:
XTrain_ = newtrFC.spoolOrangeTrainingData()
XTest_ = newtrFC.spoolOrangeTestData()

In [64]:
XTest_ = newtrFC.spoolOrangeTestData()

In [44]:
XTest_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055 entries, 0 to 2054
Data columns (total 15 columns):
business_age                    2055 non-null float64
business_first_trnx_age         2055 non-null float64
is_registered                   2055 non-null float64
is_starter                      2055 non-null float64
customer_age                    2055 non-null float64
customer_first_trnx_age         2055 non-null float64
daily_trans_instrument_count    2055 non-null float64
is_local_instrument             2055 non-null float64
paid_with_card                  2055 non-null float64
paid_with_bank                  2055 non-null float64
paid_with_other                 2055 non-null float64
amount                          2055 non-null float64
currency                        2055 non-null object
bank                            2055 non-null object
label                           2055 non-null object
dtypes: float64(12), object(3)
memory usage: 240.9+ KB


In [5]:
XTrainTable = orangedf.df2table(XTrain_)
target = Domain(XTrainTable.domain[:14], XTrainTable.domain[14])

In [6]:
XTrainTable_ = Table.from_table(target,XTrainTable)

In [9]:
from Orange.evaluation import CrossValidation, scoring

orCv2 = CrossValidation(XTrainTable_, [orange.classification.CN2Learner()], k=10)
print("Accuracy: %.3f" % scoring.CA(orCv2)[0])
print("AUC:      %.3f" % scoring.AUC(orCv2)[0])

Accuracy: 0.886
AUC:      0.940


In [10]:
learner = orange.classification.CN2Learner()
classifier = learner(XTrainTable_)

In [80]:
learner2 = orange.classification.CN2SDLearner()
classifier2 = learner2(XTrainTable_)

In [73]:
YTest = XTest_['label'].values
del XTest_['label']

In [74]:
cPred = classifier(XTest_.values)

In [75]:
output = []
for cl in YTest:
    if cl == 'legit':
        output = np.append(output, 1)
    else :
        output = np.append(output, 0)

In [76]:
print("Accuracy: ", metrics.accuracy_score(output, cPred))

Accuracy:  0.4968369829683698


In [83]:
c2Pred = classifier2(XTest_.values)

In [85]:
print("Accuracy: ", metrics.accuracy_score(output, c2Pred))

Accuracy:  0.8583941605839416


In [81]:
c_values = XTrainTable_.domain.class_var.values

In [88]:
clMatrix = metrics.confusion_matrix(output, c2Pred).ravel()
print("CN2-SD Confusion Matrix : \n{TP : %d, FN : %d,\n FP : %d, TN : %d}" % (clMatrix[0], clMatrix[1], clMatrix[2], clMatrix[3]))

CN2-SD Confusion Matrix : 
{TP : 944, FN : 90,
 FP : 201, TN : 820}


In [89]:
print("CN2-SD Classification Report : \n", metrics.classification_report(output, c2Pred, target_names=c_values))

CN2-SD Classification Report : 
               precision    recall  f1-score   support

       fraud       0.82      0.91      0.87      1034
       legit       0.90      0.80      0.85      1021

   micro avg       0.86      0.86      0.86      2055
   macro avg       0.86      0.86      0.86      2055
weighted avg       0.86      0.86      0.86      2055



In [90]:
fpr, tpr, thresh = metrics.roc_curve(output, c2Pred)
auc = metrics.roc_auc_score(output, c2Pred)

In [91]:
print(fpr, tpr, thresh, auc)

[0.         0.08704062 1.        ] [0.         0.80313418 1.        ] [2 1 0] 0.8580467816094131
